<a href="https://colab.research.google.com/github/francoisdoanp/MLTBP/blob/master/Project_MLv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine learning - Final Project**

# Turbofan engine degradation dataset (NASA)

# Data Preparation

**Importing necessary packages**



In [0]:

import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from sklearn.preprocessing import PolynomialFeatures
from keras import regularizers
import keras.backend as k

**Importing the Turbofan engine degradation dataset.**

**Files are located in the following Github repository: https://github.com/francoisdoanp/MLTBP**

We have 4 training datasets, which contains information about one hundred engines, all of the same type. Thus, we will combine the training and test data sets. 

The training and test sets have 21 columns: ID, Time (Cycles), 3 columns for operational settings and 21 sensor measurements.

The training and testing sets have the same format, while the validation sets only contain the real RUL (remaining useful life).

For more information on the data, consult the read me at the following address:https://github.com/francoisdoanp/MLTBP/blob/master/readme.txt

In [0]:
url_base = 'https://raw.githubusercontent.com/francoisdoanp/MLTBP/master/'

file_train_1 = 'train_FD001.txt'
file_train_2 = 'train_FD002.txt'
file_train_3 = 'train_FD003.txt'
file_train_4 = 'train_FD004.txt'

file_test_1 = 'test_FD001.txt'
file_test_2 = 'test_FD002.txt'
file_test_3 = 'test_FD003.txt'
file_test_4 = 'test_FD004.txt'

file_valid_1 = 'RUL_FD001.txt'
file_valid_2 = 'RUL_FD002.txt'
file_valid_3 = 'RUL_FD003.txt'
file_valid_4 = 'RUL_FD004.txt'


pt1 = pd.read_csv(url_base + file_train_1, sep=' ', header=None)
pt2 = pd.read_csv(url_base + file_train_2, sep=' ', header=None)
pt3 = pd.read_csv(url_base + file_train_3, sep=' ', header=None)
pt4 = pd.read_csv(url_base + file_train_4, sep=' ', header=None)

pte1 = pd.read_csv(url_base + file_test_1, sep=' ', header=None)
pte2 = pd.read_csv(url_base + file_test_2, sep=' ', header=None)
pte3 = pd.read_csv(url_base + file_test_3, sep=' ', header=None)
pte4 = pd.read_csv(url_base + file_test_4, sep=' ', header=None)

pv1 = pd.read_csv(url_base + file_valid_1, header=None)
pv2 = pd.read_csv(url_base + file_valid_2, header=None)
pv3 = pd.read_csv(url_base + file_valid_3, header=None)
pv4 = pd.read_csv(url_base + file_valid_4, header=None)


# Updating ids

pt2[0] = pt2[0].apply(lambda x: x+100)
pt3[0] = pt3[0].apply(lambda x: x+360)
pt4[0] = pt4[0].apply(lambda x: x+460)

pte2[0] = pte2[0].apply(lambda x: x+100)
pte3[0] = pte3[0].apply(lambda x: x+359)
pte4[0] = pte4[0].apply(lambda x: x+459)


# Joining the dataframes

train_pd = pd.concat([pt1,pt2,pt3,pt4]).reset_index(drop=True)
test_pd = pd.concat([pte1,pte2,pte3,pte4]).reset_index(drop=True)
valid_pd = pd.concat([pv1,pv2,pv3,pv4], ignore_index=True)

train_pd = train_pd.drop(train_pd.columns[[26,27]], axis='columns')
test_pd = test_pd.drop(test_pd.columns[[26,27]], axis='columns')


# Assigning labels to Dataframe's columns based on the Readme

train_pd.columns = ['id', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
test_pd.columns = ['id', 'Time (Cycles)', 'OS1', 'OS2', 'OS3', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21']
valid_pd.columns = ['RUL']

#Loading scaler

scaler = StandardScaler()
  

**Adding variables Conditons and fault mode**

Note:

**Condition (ONE)** and **Fault ONE** are binary variables.

When Condition(ONE) = 1 (true), it means that the condition is at Sea Level

When Condition(ONE) = 0 (false), it means NO, the condition IS NOT AT SEA LEVEL, and thus is the  second condition; SIX.

When Fault ONE = 1 (true), it means that the fault modes is one (HPC Degradation)

When Fault ONE = 0 (false), it means that the fault mode is TWO (HPC Degradation and Fan degradation)

In [0]:
# Adding variables Condition and fault modes

def value_condition_train(row):
  if (row['id'] <= 100):
    return 1
  elif (row['id'] <= 360) & (row['id'] > 100):
    return 0
  elif (row['id'] <= 460) & (row['id'] > 360):
    return 1
  else:
    return 0
  
def value_fault_train(row):
  if (row['id'] <= 100):
    return 1
  elif (row['id'] <= 360) & (row['id'] > 100):
    return 1
  elif (row['id'] <= 460) & (row['id'] > 360):
    return 0
  else:
    return 0
  
def value_condition_test(row):
  if (row['id'] <= 100):
    return 1
  elif (row['id'] <= 359) & (row['id'] > 100):
    return 0
  elif (row['id'] <= 459) & (row['id'] > 359):
    return 1
  else:
    return 0
  
def value_fault_test(row):
  if (row['id'] <= 100):
    return 1
  elif (row['id'] <= 359) & (row['id'] > 100):
    return 1
  elif (row['id'] <= 459) & (row['id'] > 359):
    return 0
  else:
    return 0


train_pd['Condition (One)'] = train_pd.apply(value_condition_train, axis=1)
train_pd['Fault ONE'] = train_pd.apply(value_fault_train,axis=1)

test_pd['Condition (One)'] = test_pd.apply(value_condition_test, axis=1)
test_pd['Fault ONE'] = test_pd.apply(value_fault_test,axis=1)

display(train_pd)

At this stage, we create the truth remaining useful life (RUL) for the training set.

Important note: In the training set, the last Cycle (represented in the table by 'Time (Cycles)') is when the engine is considered unusable. However, in the test set, the last cycle IS NOT when the engine is considered unusable. It will fail at a later time. Thus, in the valid_pd, we have the true RUL. 

In [0]:
#Adding column for remaining useful life (RUL)

y_train = pd.DataFrame(train_pd.groupby(['id'])['Time (Cycles)'].max())

train_pd = pd.merge(train_pd,y_train, on='id')
train_pd['RUL'] = train_pd['Time (Cycles)_y'] - train_pd['Time (Cycles)_x']
train_pd = train_pd.drop('Time (Cycles)_y',1)
train_pd = train_pd.rename(columns = {'Time (Cycles)_x':'Time (Cycles)'})

y_train = train_pd.iloc[:,28]
y_train_id = train_pd.iloc[:,[0,28]]


In [0]:
def model_score(y_true, y_pred):
  pred_df = pd.DataFrame(y_pred)
  test_err = pd.concat([y_true,pred_df], axis=1, ignore_index=True)
  test_err.columns = ['RUL', 'Pred_RUL']
  a1 = 10
  a2 = 13
  score=0

  for index, row in test_err.iterrows():
    d = row['Pred_RUL'] - row['RUL']
    if d < 0:
      score += np.expm1(-(d/a1))-1
    else:
      score += np.expm1(d/a2)-1

  return score

**Preparing data**

In [0]:
# Scaling data

train_pd_scaled = train_pd.copy()
train_pd_scaled.iloc[:,2:26] = scaler.fit_transform(train_pd.iloc[:,2:26])

test_pd_scaled = test_pd.copy()
test_pd_scaled.iloc[:,2:26] = scaler.fit_transform(test_pd.iloc[:,2:26])

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)

print(train_pd_scaled)


# **Dataset with feature engineering (Rolling Average)**

In [0]:
train_RA_pd = train_pd_scaled.copy()
period_length=10


# Rolling average for training data

rolling_avg1 = train_pd_scaled.groupby('id')["OS1"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["OS1_rolling_avg"] = rolling_avg1

rolling_avg2 = train_pd_scaled.groupby('id')["OS2"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["OS2_rolling_avg"] = rolling_avg2

rolling_avg3 = train_pd_scaled.groupby('id')["OS3"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["OS3_rolling_avg"] = rolling_avg3

rolling_avg4 = train_pd_scaled.groupby('id')["S1"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S1_rolling_avg"] = rolling_avg4

rolling_avg5 = train_pd_scaled.groupby('id')["S2"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S2_rolling_avg"] = rolling_avg5

rolling_avg6 = train_pd_scaled.groupby('id')["S3"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S3_rolling_avg"] = rolling_avg6

rolling_avg7 = train_pd_scaled.groupby('id')["S4"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S4_rolling_avg"] = rolling_avg7

rolling_avg8 = train_pd_scaled.groupby('id')["S5"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S5_rolling_avg"] = rolling_avg8

rolling_avg9 = train_pd_scaled.groupby('id')["S6"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S6_rolling_avg"] = rolling_avg9

rolling_avg10 = train_pd_scaled.groupby('id')["S7"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S7_rolling_avg"] = rolling_avg10

rolling_avg11 = train_pd_scaled.groupby('id')["S8"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S8_rolling_avg"] = rolling_avg11

rolling_avg12 = train_pd_scaled.groupby('id')["S9"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S9_rolling_avg"] = rolling_avg12

rolling_avg13 = train_pd_scaled.groupby('id')["S10"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S10_rolling_avg"] = rolling_avg13

rolling_avg14 = train_pd_scaled.groupby('id')["S11"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S11_rolling_avg"] = rolling_avg14

rolling_avg15 = train_pd_scaled.groupby('id')["S12"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S12_rolling_avg"] = rolling_avg15

rolling_avg16 = train_pd_scaled.groupby('id')["S13"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S13_rolling_avg"] = rolling_avg16

rolling_avg17 = train_pd_scaled.groupby('id')["S14"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S14_rolling_avg"] = rolling_avg17

rolling_avg18 = train_pd_scaled.groupby('id')["S15"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S15_rolling_avg"] = rolling_avg18

rolling_avg19 = train_pd_scaled.groupby('id')["S16"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S16_rolling_avg"] = rolling_avg19

rolling_avg20 = train_pd_scaled.groupby('id')["S17"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S17_rolling_avg"] = rolling_avg20

rolling_avg21 = train_pd_scaled.groupby('id')["S18"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S18_rolling_avg"] = rolling_avg21

rolling_avg22 = train_pd_scaled.groupby('id')["S19"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S19_rolling_avg"] = rolling_avg22

rolling_avg23 = train_pd_scaled.groupby('id')["S20"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S20_rolling_avg"] = rolling_avg23

rolling_avg24 = train_pd_scaled.groupby('id')["S21"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
train_RA_pd["S21_rolling_avg"] = rolling_avg24

train_RA_pd.dropna(inplace=True)

# Rolling average for test data

test_RA_pd = test_pd_scaled.copy()
period_length=10


rolling_avg1 = test_pd_scaled.groupby('id')["OS1"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["OS1_rolling_avg"] = rolling_avg1

rolling_avg2 = test_pd_scaled.groupby('id')["OS2"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["OS2_rolling_avg"] = rolling_avg2

rolling_avg3 = test_pd_scaled.groupby('id')["OS3"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["OS3_rolling_avg"] = rolling_avg3

rolling_avg4 = test_pd_scaled.groupby('id')["S1"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S1_rolling_avg"] = rolling_avg4

rolling_avg5 = test_pd_scaled.groupby('id')["S2"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S2_rolling_avg"] = rolling_avg5

rolling_avg6 = test_pd_scaled.groupby('id')["S3"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S3_rolling_avg"] = rolling_avg6

rolling_avg7 = test_pd_scaled.groupby('id')["S4"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S4_rolling_avg"] = rolling_avg7

rolling_avg8 = test_pd_scaled.groupby('id')["S5"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S5_rolling_avg"] = rolling_avg8

rolling_avg9 = test_pd_scaled.groupby('id')["S6"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S6_rolling_avg"] = rolling_avg9

rolling_avg10 = test_pd_scaled.groupby('id')["S7"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S7_rolling_avg"] = rolling_avg10

rolling_avg11 = test_pd_scaled.groupby('id')["S8"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S8_rolling_avg"] = rolling_avg11

rolling_avg12 = test_pd_scaled.groupby('id')["S9"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S9_rolling_avg"] = rolling_avg12

rolling_avg13 = test_pd_scaled.groupby('id')["S10"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S10_rolling_avg"] = rolling_avg13

rolling_avg14 = test_pd_scaled.groupby('id')["S11"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S11_rolling_avg"] = rolling_avg14

rolling_avg15 = test_pd_scaled.groupby('id')["S12"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S12_rolling_avg"] = rolling_avg15

rolling_avg16 = test_pd_scaled.groupby('id')["S13"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S13_rolling_avg"] = rolling_avg16

rolling_avg17 = test_pd_scaled.groupby('id')["S14"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S14_rolling_avg"] = rolling_avg17

rolling_avg18 = test_pd_scaled.groupby('id')["S15"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S15_rolling_avg"] = rolling_avg18

rolling_avg19 = test_pd_scaled.groupby('id')["S16"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S16_rolling_avg"] = rolling_avg19

rolling_avg20 = test_pd_scaled.groupby('id')["S17"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S17_rolling_avg"] = rolling_avg20

rolling_avg21 = test_pd_scaled.groupby('id')["S18"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S18_rolling_avg"] = rolling_avg21

rolling_avg22 = test_pd_scaled.groupby('id')["S19"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S19_rolling_avg"] = rolling_avg22

rolling_avg23 = test_pd_scaled.groupby('id')["S20"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S20_rolling_avg"] = rolling_avg23

rolling_avg24 = test_pd_scaled.groupby('id')["S21"] \
    .apply(lambda x: x.shift().rolling(period_length, min_periods=10)
    .mean())
test_RA_pd["S21_rolling_avg"] = rolling_avg24

test_RA_pd.dropna(inplace=True)

In [0]:
# Removing RUL and id columns, as we do not want these features to be in our predictors

train_pd_ra = train_RA_pd.copy()
train_pd_ra = train_pd_ra.drop(['id', 'RUL','Time (Cycles)'],axis=1)



In [0]:
# Removing RUL and id columns, as we do not want these features to be in our predictors

train_pd_lm = train_pd_scaled.copy()
train_pd_lm = train_pd_lm.drop(['id', 'RUL', 'Time (Cycles)'],axis=1)

# Keeping only last time cycle for each id

idx = test_pd_scaled.groupby(['id'])['Time (Cycles)'].transform(max) == test_pd_scaled['Time (Cycles)']
test_pd_lm = test_pd_scaled[idx]

idx4 = test_RA_pd.groupby(['id'])['Time (Cycles)'].transform(max) == test_RA_pd['Time (Cycles)']
test_pd_ra = test_RA_pd[idx4]

# Removing id column

test_pd_lm = test_pd_lm.drop(['id', 'Time (Cycles)'], axis=1)
test_pd_ra = test_pd_ra.drop(['id', 'Time (Cycles)'], axis=1)

y_train_ra = y_train_id.groupby('id').apply(lambda group: group.iloc[10:]).reset_index(drop=True)
y_train_ra = y_train_ra.drop(['id'], axis=1)


# **Model 3: Neural Networks**

# **Model 3.1: Neural Network**

In [0]:
# Building Model

model2 = Sequential()
model2.add(Dense(100,  activation='relu', input_shape=[len(train_pd_lm.keys())], kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01),  activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model2.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))

model2.add(Dense(1))

model2.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

print(model2.summary())

In [33]:
# Fitting model

NNmodel2 = model2.fit(train_pd_lm, y_train, epochs=100, batch_size=64, validation_split=0.05, verbose=1,callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min')])
#NNmodel2 = model2.fit(train_pd_lm, y_train, epochs=10, batch_size=200, validation_split=0.05, verbose=1)

Train on 152341 samples, validate on 8018 samples
Epoch 1/500
152341/152341 [==============================] - 10s 63us/step - loss: 5051.5475 - mean_absolute_error: 46.5742 - val_loss: 3431.8512 - val_mean_absolute_error: 40.4647
Epoch 2/500
152341/152341 [==============================] - 9s 59us/step - loss: 3464.6898 - mean_absolute_error: 40.9135 - val_loss: 3495.5296 - val_mean_absolute_error: 43.5367
Epoch 3/500
152341/152341 [==============================] - 9s 59us/step - loss: 3277.7418 - mean_absolute_error: 40.0796 - val_loss: 3662.7956 - val_mean_absolute_error: 43.9263
Epoch 4/500
152341/152341 [==============================] - 9s 59us/step - loss: 3185.6948 - mean_absolute_error: 39.6270 - val_loss: 3416.0031 - val_mean_absolute_error: 43.8641
Epoch 5/500
152341/152341 [==============================] - 9s 60us/step - loss: 3120.9436 - mean_absolute_error: 39.2643 - val_loss: 3882.3932 - val_mean_absolute_error: 50.1131
Epoch 6/500
152341/152341 [======================

In [0]:
# Testing on validation set

y_pred_nn = model2.predict(test_pd_lm, batch_size=20, verbose=1)

result_nn = metrics.mean_absolute_error(y_pred_nn, valid_pd)

print(f'The mean absoute error of the Neural Network on the test test is {result_nn}.')

score_nn = model_score(valid_pd, y_pred_nn)

print(f'The score of the linear model is: {(score_nn)}')


707/707 [==============================] - 0s 63us/step
The mean absoute error of the Neural Network on the test test is 25.535554069778033.
The score of the linear model is: 1236077.2241536288


# **Model 3.2: Neural Network with Feature Engineering**

In [0]:
# Building model

model4 = Sequential()
model4.add(Dense(100,  activation='relu', input_shape=[len(train_pd_ra.keys())], kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01),  activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model4.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))

model4.add(Dense(1))

model4.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

print(model4.summary())

In [0]:
# Fitting Model

NNmodel4 = model4.fit(train_pd_ra, y_train_ra, epochs=100, batch_size=64, validation_split=0.05, verbose=1,callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min')])

Train on 145605 samples, validate on 7664 samples
Epoch 1/100
145605/145605 [==============================] - 10s 67us/step - loss: 4796.6725 - mean_absolute_error: 44.6032 - val_loss: 3862.8068 - val_mean_absolute_error: 45.2265
Epoch 2/100
145605/145605 [==============================] - 9s 60us/step - loss: 3200.0164 - mean_absolute_error: 38.7590 - val_loss: 3336.7090 - val_mean_absolute_error: 42.9633
Epoch 3/100
145605/145605 [==============================] - 9s 59us/step - loss: 2984.1922 - mean_absolute_error: 37.6988 - val_loss: 2963.6625 - val_mean_absolute_error: 39.5110
Epoch 4/100
145605/145605 [==============================] - 9s 60us/step - loss: 2888.1090 - mean_absolute_error: 37.1764 - val_loss: 3523.4667 - val_mean_absolute_error: 42.8255
Epoch 5/100
145605/145605 [==============================] - 9s 59us/step - loss: 2833.2082 - mean_absolute_error: 36.8753 - val_loss: 2816.0754 - val_mean_absolute_error: 38.3688
Epoch 6/100
145605/145605 [======================

In [0]:
# Testing on validation set

y_pred_nn2 = model4.predict(test_pd_ra, batch_size=20, verbose=1)

result_nn2 = metrics.mean_absolute_error(y_pred_nn2, valid_pd)

print(f'The mean absoute error of the Neural Network on the test test is {result_nn2}.')

score_nn2 = model_score(valid_pd, y_pred_nn2)

print(f'The score of the linear model is: {(score_nn2)}')

707/707 [==============================] - 0s 58us/step
The mean absoute error of the Neural Network on the test test is 26.99157351516767.
The score of the linear model is: 786005.6983353865


# **Model 3.2: Neural Network with LSTM architecture**

In [0]:
# Using Azure tutorial for predictive maintenance for data manipulation
# Reference: https://github.com/Azure/lstms_for_predictive_maintenance/blob/master/Deep%20Learning%20Basics%20for%20Predictive%20Maintenance.ipynb

# Picking a sequence length - This will be the window of time in which the LSTM will gather data from

sequence_length = 50


# This function will reshape our data so it can be usable with Keras (Samples, time window, features)
def gen_sequence(id_df, seq_length,seq_cols):
  data_array = id_df[seq_cols].values
  num_elements = data_array.shape[0]
  for start,stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
    yield data_array[start:stop,:]

# Reference column names

sensor_cols = ['S' + str(i) for i in range(1,22)]
sequence_cols = ['Time (Cycles)', 'OS1', 'OS2', 'OS3']
other_cols = ['Condition (One)', 'Fault ONE']
sequence_cols.extend(sensor_cols)
sequence_cols.extend(other_cols)

# Generating sequences

seq_gen = (list(gen_sequence(train_pd_scaled[train_pd_scaled['id']==id], sequence_length,sequence_cols))
          for id in train_pd_scaled['id'].unique())

# Generate sequences and convert to numpy array

seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

def gen_labels(id_df, seq_length, label):
  data = id_df[label].values
  num_elements = data.shape[0]
  return data[seq_length:num_elements,:]

label_gen = [gen_labels(train_pd_scaled[train_pd_scaled['id']==id], sequence_length, ['RUL'])
            for id in train_pd_scaled['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
print(label_array.shape)

(124909, 50, 27)
(124909, 1)


In [0]:
# Building the RNN

nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(input_shape=(sequence_length, nb_features), units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(input_shape=(sequence_length, nb_features), units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(input_shape=(sequence_length, nb_features), units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(input_shape=(sequence_length, nb_features), units=50, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out))
model.add(Activation('linear'))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

print(model.summary())


NameError: ignored

In [0]:
# Fitting the RNN

model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=1, callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')])

NameError: ignored

In [0]:
# Adding RUL to test set

truth_nn= valid_pd.copy()
max_hid = pd.DataFrame(test_pd.groupby('id')['Time (Cycles)'].max()).reset_index()
max_hid.columns = ['id','max']
truth_nn.columns = ['truth']
truth_nn['id'] = truth_nn.index +1
truth_nn['truth'] = truth_nn['truth'] + max_hid['max']

test_pd_nn = test_pd.copy()
test_pd_nn = test_pd_nn.merge(truth_nn, on=['id'], how='left')
test_pd_nn['RUL'] = test_pd_nn['truth'] - test_pd_nn['Time (Cycles)']
test_pd_nn.drop('truth', axis=1, inplace=True)

print(test_pd_nn)

In [0]:
# Preparing test set

seq_array_test = [test_pd_nn[test_pd_nn['id']==id][sequence_cols].values[-sequence_length:]
                 for id in test_pd_nn['id'].unique() if len(test_pd_nn[test_pd_nn['id']==id]) >= sequence_length]

seq_array_test = np.asarray(seq_array_test).astype(np.float32)

y_mask = [len(test_pd_nn[test_pd_nn['id']==id]) >= sequence_length for id in test_pd_nn['id'].unique()]

label_array_test = test_pd_nn.groupby('id')['RUL'].nth(-1)[y_mask].values
label_array_test = label_array_test.reshape(label_array_test.shape[0],1).astype(np.float32)

print(seq_array_test.shape)
print(label_array_test.shape)

(653, 50, 27)
(653, 1)


In [0]:
# Fitting model on test set

y_pred_lstm = model.predict(seq_array_test, batch_size=32, verbose=1)

result_lstm =  metrics.mean_absolute_error(y_pred_lstm, label_array_test)

print(f'The mean absolute error for the LSTM on the test set is {result_lstm}.' )

score_lstm = model_score(valid_pd, y_pred_lstm)

print(f'The score of the linear model is: {(score_lstm)}')


653/653 [==============================] - 3s 4ms/step
The mean absolute error for the LSTM on the test set is 41.70280838012695.
The score of the linear model is: nan


## **Neural Network models with custom loss function**

This function will penalize heavily predictions that are greater than the true RUL. In practice, there could be grave consequences to overshooting. Thus, we will penalize by two times.

In [0]:
def customLossMSE(true,pred):
  diff = pred - true

  greater= k.greater(diff,0)
  greater= k.cast(greater, k.floatx())
  greater += 1

  return k.mean(greater*k.square(diff))

def customLossMAE(true,pred):
  diff = pred - true

  greater= k.greater(diff,0)
  greater= k.cast(greater, k.floatx())
  greater += 1

  return k.mean(greater*k.absolute(diff))




In [0]:
# Building NN Model

model3 = Sequential()
model3.add(Dense(100,  activation='relu', input_shape=[len(train_pd_lm.keys())], kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01),  activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model3.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))

model3.add(Dense(1))

model3.compile(loss=customLossMSE, optimizer='rmsprop', metrics=['mae'])

print(model3.summary())

In [0]:
# Fitting model

model3.fit(train_pd_lm, y_train, epochs=100, batch_size=64, validation_split=0.05, verbose=1,callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min')])

In [0]:
# Testing on validation set

y_pred_nn_cus = model3.predict(test_pd_lm, batch_size=20, verbose=1)

result_nn_cus = metrics.mean_absolute_error(y_pred_nn_cus, valid_pd)

print(f'The mean absoute error of the Neural Network on the test test is {result_nn_cus}.')

score_nn_cus = model_score(valid_pd, y_pred_nn_cus)

print(f'The score of the linear model is: {(score_nn_cus)}')

707/707 [==============================] - 2s 2ms/step
The mean absoute error of the Neural Network on the test test is 24.387866405988852.
The score of the linear model is: 350476.33434362814


In [0]:
# Building model with feature engineering

model5 = Sequential()
model5.add(Dense(100,  activation='relu', input_shape=[len(train_pd_ra.keys())], kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01),  activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))
model5.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)))

model5.add(Dense(1))

model5.compile(loss=customLossMSE, optimizer='rmsprop', metrics=['mae'])

print(model5.summary())

In [0]:
# Fitting Model

model5.fit(train_pd_ra, y_train_ra, epochs=100, batch_size=64, validation_split=0.05, verbose=1,callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min')])

In [0]:
# Testing on validation set

y_pred_nn_cus_ra = model5.predict(test_pd_ra, batch_size=20, verbose=1)

result_nn_cus_ra = metrics.mean_absolute_error(y_pred_nn_cus_ra, valid_pd)

print(f'The mean absoute error of the Neural Network on the test test is {result_nn_cus_ra}.')

score_nn_cus_ra = model_score(valid_pd, y_pred_nn_cus_ra)

print(f'The score of the linear model is: {(score_nn_cus_ra)}')

707/707 [==============================] - 0s 70us/step
The mean absoute error of the Neural Network on the test test is 26.103144309476882.
The score of the linear model is: 1552295.2335257854


In [0]:
# Building LSTM model

nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model6 = Sequential()
model6.add(LSTM(input_shape=(sequence_length, nb_features), units=100, return_sequences=True))
model6.add(Dropout(0.2))
model6.add(LSTM(input_shape=(sequence_length, nb_features), units=50, return_sequences=False))
model6.add(Dropout(0.2))

model6.add(Dense(units=nb_out))
model6.add(Activation('relu'))
model6.compile(loss=customLossMSE, optimizer='rmsprop', metrics=['mae'])

print(model6.summary())

In [0]:
model6.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=1, callbacks =[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')])

Train on 118663 samples, validate on 6246 samples
Epoch 1/100
118663/118663 [==============================] - 162s 1ms/step - loss: 10860.0463 - mean_absolute_error: 77.5889 - val_loss: 9151.1605 - val_mean_absolute_error: 72.0223
Epoch 2/100
118663/118663 [==============================] - 162s 1ms/step - loss: 7648.4114 - mean_absolute_error: 61.5949 - val_loss: 6660.0211 - val_mean_absolute_error: 59.4159
Epoch 3/100
118663/118663 [==============================] - 162s 1ms/step - loss: 5770.0928 - mean_absolute_error: 51.1364 - val_loss: 4469.0203 - val_mean_absolute_error: 46.4810
Epoch 4/100
118663/118663 [==============================] - 162s 1ms/step - loss: 4609.0389 - mean_absolute_error: 44.1671 - val_loss: 3513.7253 - val_mean_absolute_error: 39.9294
Epoch 5/100
118663/118663 [==============================] - 162s 1ms/step - loss: 4000.2422 - mean_absolute_error: 40.4408 - val_loss: 2867.8935 - val_mean_absolute_error: 35.5230
Epoch 6/100
118663/118663 [=================

In [0]:
# Fitting LSTM with custom loss

y_pred_lstm_cus = model6.predict(seq_array_test, batch_size=32, verbose=1)

result_lstm_cus =  metrics.mean_absolute_error(y_pred_lstm_cus, label_array_test)

print(f'The mean absolute error for the LSTM on the test set is {result_lstm_cus}.' )

score_lstm_cus = model_score(valid_pd, y_pred_lstm_cus)

print(f'The score of the linear model is: {(score_lstm_cus)}')

653/653 [==============================] - 0s 628us/step
The mean absolute error for the LSTM on the test set is 43.24674987792969.
The score of the linear model is: nan


In [0]:
print(y_pred_lstm)

[[84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [88.76844 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]
 [84.69474 ]

# **Testing with Softplus**